# **Patching Surface for *snappyHexMesh* Generation**
<hr>

In [2]:
from vmtk import vmtkscripts
import numpy as np
import os

In [3]:
%run '../vmtk_functions.py'
%run '../vmtk_filenames.py'

In [4]:
# Path where files are stored
aneurysmsPath = "/home/iagolessa/Documents/unesp/doctorate/data/"

In [5]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       = aneurysmsPath+"aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case14/"

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

Files saved to: /home/iagolessa/Documents/unesp/doctorate/data/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case14/


<hr>

# **Surface Orientation and Patching**

The cells below takes care of orienting the surface by translate and rotate transformations, to conform the inlet patch surface normal to the opposite z direction (this orientation was chosen to facilitate the introduction of boundary conditions at simulation time). 

The oriented surface is then 'exploded' to its constituent patches using the 'vmtkthreshold' script. First, the boundary is inspected generating a reference system file -- 'referenceSystem.dat' with the normal and center information of each cap patch (the wall has id 1 by default, and we prefer to keep it that way). The capped surface is, then, transformed and saved to disk -- 'surfaceOriented.stl' -- and finally is decomposed into its patches.

One of the cells below try to identify the inlet patch index by the patch area, because, usually, the inlet has larger area than the outlets (but is always important to be sure of the inlet id).

In [28]:
outletFileName  = 'outlet.stl'
outletFileName[0:6], outletFileName[6:10]

('outlet', '.stl')

In [46]:
%%writefile -a ../vmtk_functions.py

def patchSurface(surface, outputDir, capped=True):
    """ 
    Function to patch a surface to be used 
    in CFD by creating a mesh in snappyHexMesh.
    The function also orient the surface putting
    the inlet (selected by largest radius) center
    on the origin of the system, with its outward
    normal point to -z direction.
    """

    inletFileName  = 'inlet.stl'
    outletFileName = 'outlet.stl'
    wallFileName   = 'wall.stl'
    CellEntityIdsArray   = 'CellEntityIds'
    BoundaryRadiusArray  = 'BoundaryRadius'
    BoundaryNormalsArray = 'BoundaryNormals'
    
    origin = [0, 0, 0]
    orientation = [0, 0, -1]
    
    
    # Check if patch dir exists
    patchDir = outputDir+'patches/'
    if not os.path.isdir(patchDir):
        os.makedirs(patchDir)
#         print(patchDir+' created.')
    
    
    # Check cap condition
    if capped == False:
        surfaceCapper = vmtkscripts.vmtkSurfaceCapper()
        surfaceCapper.Surface = surface
        surfaceCapper.Method  = 'centerpoint'
        surfaceCapper.Interactive = 0
        surfaceCapper.Execute()
        
        surface = surfaceCapper.Surface

        
    # Surface boundary inspector
    # Conveting surface to mesh to get cell entity ids
    surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()
    surfaceToMesh.Surface = surface
    surfaceToMesh.Execute()

    # Inspecting
    surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()
    surfaceBoundaryInspector.Mesh = surfaceToMesh.Mesh
    surfaceBoundaryInspector.CellEntityIdsArrayName = CellEntityIdsArray
    surfaceBoundaryInspector.Execute()

    # Store patch info in python dictionary using vmtksurfacetonumpy
    vmtkToNumpy = vmtkscripts.vmtkSurfaceToNumpy()
    vmtkToNumpy.Surface = surfaceBoundaryInspector.ReferenceSystems
    vmtkToNumpy.Execute()

    dictPatchData = vmtkToNumpy.ArrayDict
    

    # Get inlet by maximum radius condition
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Get max radius and its index
    maxRadius = max(dictPatchData['PointData'][BoundaryRadiusArray])
    inletPos, = np.where( dictPatchData['PointData'][BoundaryRadiusArray] == maxRadius )

    # Number of patches
    nPatches  = len(dictPatchData['PointData'][CellEntityIdsArray])

    # Update dictPatchData with patch info
    dictPatchData['PatchType'] = np.array(nPatches*['outlet'])
    dictPatchData['PatchType'][int(inletPos)] = 'inlet'
#     dictPatchData

    wallId  = 1
    inletId = dictPatchData['PointData'][CellEntityIdsArray][int(inletPos)]

    # Counting the number of inlets and outlets
    # using the numpy.unique function: it returns an ndarray with the unique itens of the input array
    # with the return_counts=True also returns the number of appearance of each item
    patchTypes, numberOfPatches = np.unique(dictPatchData['PatchType'], return_counts=True)

    
    # Zip both patchTypes and numberOfPatches and convert to dictionary
    # with number of inlets and outlets
    patchCount = dict(zip(patchTypes, numberOfPatches))

    # Surface reorientation
    # The vmtksurfacereferencesystemtransform script takes a surface (the surfaceCapped above) 
    # and rotates and translates it conforming one of its patches (in our case the inlet patch) 
    # to a target reference system for that, it uses the output of the vtmkboundaryinspector
    surfaceTransform = vmtkscripts.vmtkSurfaceReferenceSystemTransform()
    surfaceTransform.Surface = surfaceToMesh.Surface

    # Target reference system parameters
    surfaceTransform.ReferenceOrigin  = origin  # translate to origin of system
    surfaceTransform.ReferenceNormal1 = orientation # inlet normal will coincide with -z axis orientation
    surfaceTransform.ReferenceNormal2 = orientation

    # Surface reference system
    surfaceTransform.ReferenceSystems = surfaceBoundaryInspector.ReferenceSystems
    # to get the reference systems of inlet patch
    # Note that, if there is more than one inlet, the inlet chose is the one with largest area
    surfaceTransform.ReferenceSystemId = inletId
    surfaceTransform.ReferenceSystemsIdArrayName      = CellEntityIdsArray
    surfaceTransform.ReferenceSystemsNormal1ArrayName = BoundaryNormalsArray
    surfaceTransform.ReferenceSystemsNormal2ArrayName = BoundaryNormalsArray
    surfaceTransform.Execute()

    
    # Using vmtkThreshold script to extract patches for mesh generations in snappy
    # Extracting first the wall
    extractThreshold = vmtkscripts.vmtkThreshold()
    extractThreshold.Surface = surfaceTransform.Surface
    extractThreshold.LowThreshold  = wallId
    extractThreshold.HighThreshold = wallId
    extractThreshold.SurfaceOutputFileName = patchDir+wallFileName
    extractThreshold.OutputText('Extracting surface with id '+str(wallId)+'\n')
    extractThreshold.Execute()
    extractThreshold.IOWrite()
    extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')

    # Outlet initial index (to increment) and to be used in output filename
    outletIndex = 1
    inletIndex  = 1

    # Loop to extract inlet and outlet patches
    for i in np.arange(nPatches):
        # Instantiate vmtkthreshold
        extractThreshold = vmtkscripts.vmtkThreshold()
        extractThreshold.Surface       = surfaceTransform.Surface
        extractThreshold.LowThreshold  = dictPatchData['PointData'][CellEntityIdsArray][i]
        extractThreshold.HighThreshold = dictPatchData['PointData'][CellEntityIdsArray][i]
        extractThreshold.OutputText('Extracting surface with id '+str(dictPatchData['PointData'][CellEntityIdsArray][i])+'\n')
        
        
        # Defining output file names
        if dictPatchData['PatchType'][i] == 'inlet':

            if patchCount['inlet'] == 1:
                inletFilePath = patchDir+inletFileName
            else:
                prefix = inletFileName[0:5]
                suffix = inletFileName[5:9]
                inletFilePath = patchDir + prefix + str(inletIndex) + suffix
                inletIndex += 1
                
            extractThreshold.SurfaceOutputFileName = inletFilePath

            
        elif dictPatchData['PatchType'][i] == 'outlet':
            if patchCount['outlet'] == 1:
                outletFilePath = patchDir+outletFileName
            else:
                prefix = outletFileName[0:6]
                suffix = outletFileName[6:10]
                outletFilePath = patchDir + prefix + str(outletIndex) + suffix
                outletIndex += 1

            extractThreshold.SurfaceOutputFileName = outletFilePath

        extractThreshold.Execute()
        extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')
        extractThreshold.IOWrite()

Appending to ../vmtk_functions.py


In [36]:
# Get array with pacthes info
# surface = vmtk_functions.readSurface(surfacesDir+'surfaceRemeshedSpatialVaryingResolution.stl')
surface = readSurface(surfacesDir+surfaceCappedFile)
viewSurface(surface)

Reading VTK XML surface file.
Quit renderer


In [43]:
patchSurface(surface,'/home/iagolessa/')


CellEntityId: 2
  Origin: 62.042797, 64.870956, 83.123367
  Normal: 0.050951, -0.004113, 0.998693
  Radius: 2.312869

CellEntityId: 3
  Origin: 57.510494, 60.195553, 71.737450
  Normal: -0.441277, -0.879628, 0.177565
  Radius: 1.029664

CellEntityId: 4
  Origin: 77.544365, 61.752296, 65.396210
  Normal: 0.717085, 0.511094, 0.473891
  Radius: 1.436829

Quit renderer
wrapping vtkPolyData object
converting cell data: 
converting points
converting point data: 
BoundaryNormals
BoundaryRadius
Point1Array
Point2Array
CellEntityIds
converting cell connectivity list
Extracting surface with id 1
Writing STL surface file.
Patch saved in /home/iagolessa/patches/wall.stl
Extracting surface with id 2
Patch saved in /home/iagolessa/patches/inlet.stl
Writing STL surface file.
Extracting surface with id 3
outlet .stl
Patch saved in /home/iagolessa/patches/outlet1.stl
Writing STL surface file.
Extracting surface with id 4
outlet .stl
Patch saved in /home/iagolessa/patches/outlet2.stl
Writing STL surfac

<hr>

In [ ]:
# Capping the surface
surfaceCapper = vmtkscripts.vmtkSurfaceCapper()
surfaceCapper.Surface = surface
surfaceCapper.Method = 'centerpoint'
surfaceCapper.Interactive = 0
surfaceCapper.Execute()

In [ ]:
# Surface boundary inspector
# Conveting surface to mesh to get cell entity ids
surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()
surfaceToMesh.Surface = surfaceCapper.Surface
surfaceToMesh.Execute()

# Inspecting
surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()
surfaceBoundaryInspector.Mesh = surfaceToMesh.Mesh
surfaceBoundaryInspector.CellEntityIdsArrayName = surfaceCapper.CellEntityIdsArrayName
surfaceBoundaryInspector.Execute()

# Store patch info in python dictionary using vmtksurfacetonumpy
vmtkToNumpy = vmtkscripts.vmtkSurfaceToNumpy()
vmtkToNumpy.Surface = surfaceBoundaryInspector.ReferenceSystems
vmtkToNumpy.Execute()

dictPatchData = vmtkToNumpy.ArrayDict

In [ ]:
# # Build condition array where centers are not equal to inlet center
# # therefore, outlet centers
# inletBarycenterArray = dictPatchData['Points'][int(inletPos)]
# notInlet = (dictPatchData['Points'] != inletBarycenterArray)

# # Inlet and outlet centers
# # inletBarycenters  = dictPatchData['Points'][int(index)].tolist()
# outletBarycenters = np.extract(notInlet, dictPatchData['Points']).tolist()

In [ ]:
# Get inlet by maximum radius condition
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Get max radius and its index
maxRadius = max(dictPatchData['PointData']['BoundaryRadius'])
inletPos, = np.where( dictPatchData['PointData']['BoundaryRadius'] == maxRadius )

# Number of patches
nPatches  = len(dictPatchData['PointData']['CellEntityIds'])

# Update dictPatchData with patch info
dictPatchData['PatchType'] = np.array(nPatches*['outlet'])
dictPatchData['PatchType'][int(inletPos)] = 'inlet'
dictPatchData

In [ ]:
wallId  = 1
inletId = dictPatchData['PointData']['CellEntityIds'][int(inletPos)]

lastId = np.max(dictPatchData['PointData']['CellEntityIds'])

wallId, inletId, lastId

In [ ]:
# Counting the number of inlets and outlets
# using the numpy.unique function: it returns an ndarray with the unique itens of the input array
# with the return_counts=True also returns the number of appearance of each item
patchTypes, numberOfPatches = np.unique(dictPatchData['PatchType'], return_counts=True)

# Zip both patchTypes and numberOfPatches and convert to dictionary
# with number of inlets and outlets
patchCount = dict(zip(patchTypes, numberOfPatches))
patchCount

In [ ]:
# Surface reorientation
# It is better to reorient the final capped surface before sending to mesh in SnappyHexMesh
# The vmtksurfacereferencesystemtransform script takes a surface (the surfaceCapped above) 
# and rotates and translates it conforming one of its patches (in our case the inlet patch) to a target reference system
# for that, it uses the output of the vtmkboundaryinspector

surfaceTransform = vmtkscripts.vmtkSurfaceReferenceSystemTransform()

# or use the just created surfaceCapped
surfaceTransform.Surface = surfaceToMesh.Surface

# Target reference system parameters
surfaceTransform.ReferenceOrigin  = [0, 0, 0]  # translate to origin of system
surfaceTransform.ReferenceNormal1 = [0, 0, -1] # inlet normal will coincide with -z axis orientation
surfaceTransform.ReferenceNormal2 = [0, 0, -1]

# Surface reference system
surfaceTransform.ReferenceSystems = surfaceBoundaryInspector.ReferenceSystems

# to get the reference systems of inlet patch
# Note that, if there is more than one inlet, the inlet chose is the one with largest area
surfaceTransform.ReferenceSystemId = inletId

surfaceTransform.ReferenceSystemsIdArrayName      = surfaceCapper.CellEntityIdsArrayName
surfaceTransform.ReferenceSystemsNormal1ArrayName = 'BoundaryNormals'
surfaceTransform.ReferenceSystemsNormal2ArrayName = 'BoundaryNormals'

surfaceTransform.SurfaceOutputFileName = surfacesDir+surfaceOrientedFile
surfaceTransform.Execute()
surfaceTransform.IOWrite()

# Debug
# surfaceTransform.PrintInputMembers()
# surfaceTransform.PrintOutputMembers()

In [ ]:
# Check if patch dir exists
# If it not exists, create it
# the patch dir will contains the surface patch files for meshing (wall, inlet and outlet(s)) 

patchDir = surfacesDir+'patches/'

if not os.path.isdir(patchDir):
    os.makedirs(patchDir)
    print(patchDir+' created.')

In [ ]:
# Using vmtkThreshold script to extract patches for mesh generations in snappy

# Surface name to decompose if needs to be read before
# if you uncomment these next two lines, dont forget to change line 22 below
# surfaceOriented = readSurface(surfacesDir+surfaceCappedFile)

# File names for patches
wallFileName = patchDir+'wall.stl'

# Extracting first the wall
extractThreshold = vmtkscripts.vmtkThreshold()

extractThreshold.Surface = surfaceTransform.Surface
extractThreshold.LowThreshold  = wallId
extractThreshold.HighThreshold = wallId
extractThreshold.SurfaceOutputFileName = wallFileName

print('Extracting surface with id ', wallId)

extractThreshold.Execute()
extractThreshold.IOWrite()
extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')

# Outlet initial index (to increment) and to be used in output filename
outletIndex = 1
inletIndex  = 1

# Loop to extract inlet and outlet patches
for i in np.arange(nPatches):
    
    print('Extracting surface with id ', dictPatchData['PointData']['CellEntityIds'][i])
    
    # Instantiate vmtkthreshold
    extractThreshold = vmtkscripts.vmtkThreshold()
    extractThreshold.Surface       = surfaceTransform.Surface
    extractThreshold.LowThreshold  = dictPatchData['PointData']['CellEntityIds'][i]
    extractThreshold.HighThreshold = dictPatchData['PointData']['CellEntityIds'][i]
    
    # Defining output file names
    if dictPatchData['PatchType'][i] == 'inlet':
        
        if patchCount['inlet'] == 1:
            inletFileName = patchDir+'inlet.stl'
        else:
            inletFileName = patchDir+'inlet'+str(inletIndex)+'.stl'
            inletIndex += 1
            
        # Attribute surface output file name
        print('Patch with id '+str(dictPatchData['PointData']['CellEntityIds'][i])+' is an inlet. Defining file name: '+inletFileName) # debugging
        
        extractThreshold.SurfaceOutputFileName = inletFileName
        
    elif dictPatchData['PatchType'][i] == 'outlet':
        
        if patchCount['outlet'] == 1:
            outletFileName = patchDir+'outlet.stl'
            
        else:
            outletFileName = patchDir+'outlet'+str(outletIndex)+'.stl'
            outletIndex += 1
            
        print('Patch with id '+str(dictPatchData['PointData']['CellEntityIds'][i])+' is an outlet. Defining file name: '+outletFileName) # debugging
        
        extractThreshold.SurfaceOutputFileName = outletFileName
    
    extractThreshold.Execute()
    extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')
    extractThreshold.IOWrite()
    print('\n')

<hr>
The two cells below save the oriented surface and its Voronoi diagram to the parent vessel reconstruction procedure 

In [ ]:
# Final surface for parent vessel reconstruction
caseId = 'case11'

if not os.path.isdir(parentVesselDir+caseId):
    os.makedirs(parentVesselDir+caseId)
    
surfaceForParentVessel = parentVesselDir+caseId+"/"+caseId+"_model.vtp"
print(surfaceForParentVessel)
vmtk_functions.writeSurface(surfaceTransform.Surface, surfaceForParentVessel, "binary")

In [ ]:
# Compute Voronoi Diagram Separately for parent vessel reconstruction
voronoiDiagram = vmtkscripts.vmtkDelaunayVoronoi()

voronoiDiagram.Surface = surfaceTransform.Surface
# voronoiDiagram.SurfaceInputFileName = surfaceInputFile #casePath+"surfaces/surfaceRemeshed.vtp" 
# voronoiDiagram.IORead()
voronoiDiagram.CheckNonManifold = 1
voronoiDiagram.RemoveSubresolutionTetrahedra = 1
# voronoiDiagram.DelaunayTessellationOutputFileName = casePath+"surfaces/delaunayTesselation.vtp"
voronoiDiagram.VoronoiDiagramOutputFileName = parentVesselDir+caseId+"/"+caseId+"_voronoi.vtp"
voronoiDiagram.Execute()
voronoiDiagram.IOWrite()